Homework 5: Neural Language Models  (& 🎃 SpOoKy 👻 authors 🧟 data) - Task 3
---

Names & Sections
----
Names: __Tijana Cosic (4120) & Max Breslauer-Friedman (4120)__

Task 3: Feedforward Neural Language Model (60 points)
--------------------------

For this task, you will create and train neural LMs for both your word-based embeddings and your character-based ones. You should write functions when appropriate to avoid excessive copy+pasting.

### a) First, encode  your text into integers (5 points)

In [1]:
# Importing utility functions from Keras
import keras
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical

# necessary
from keras.models import Sequential
from keras.layers import Dense

# optional
# from keras.layers import Dropout

# if you want fancy progress bars
from tqdm import notebook
from IPython.display import display

# your other imports here
import time
import numpy as np
import neurallm_utils as nutils

[nltk_data] Downloading package punkt to /Users/maxnbf/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
# load in necessary data
from gensim.models import KeyedVectors

# abstract into util functions
NGRAM = 3 # The ngram language model you want to train
EMBEDDING_SAVE_FILE_WORD = "spooky_embedding_word.txt" # The file to save your word embeddings to
EMBEDDING_SAVE_FILE_CHAR = "spooky_embedding_char.txt" # The file to save your word embeddings to
TRAIN_FILE = 'spooky_author_train.csv' # The file to train your language model on

# reads in the data tokenized by char and by word
data_by_char = nutils.read_file_spooky(TRAIN_FILE, NGRAM, by_character=True)
data_by_word = nutils.read_file_spooky(TRAIN_FILE, NGRAM, by_character=False)

In [3]:
# constants you may find helpful. Edit as you would like.
EMBEDDINGS_SIZE = 50
NGRAM = 3 # The ngram language model you want to train

In [4]:
# Initialize a Tokenizer and fit on your data
# do this for both the word and character data

# It is used to vectorize a text corpus. Here, it just creates a mapping from 
# word to a unique index. (Note: Indexing starts from 0)
# Example:
# tokenizer = Tokenizer()
# tokenizer.fit_on_texts(data)
# encoded = tokenizer.texts_to_sequences(data)

char_tokenizer = Tokenizer()
char_tokenizer.fit_on_texts(data_by_char)
char_encoded = char_tokenizer.texts_to_sequences(data_by_char)

word_tokenizer = Tokenizer()
word_tokenizer.fit_on_texts(data_by_word)
word_encoded = word_tokenizer.texts_to_sequences(data_by_word)

In [5]:
# print out the size of the word index for each of your tokenizers
# this should match what you calculated in Task 2 with your embeddings
print("Size of word index for character tokenizer: ", len(char_tokenizer.word_index))
print("Size of word index for word tokenizer: ", len(word_tokenizer.word_index))

Size of word index for character tokenizer:  60
Size of word index for word tokenizer:  25374


### b) Next, prepare the sequences to train your model from text (5 points)

#### Fixed n-gram based sequences

In [6]:
def generate_ngram_training_samples(encoded: list, ngram: int) -> list:
    '''
    Takes the encoded data (list of lists) and 
    generates the training samples out of it.
    Parameters:
    up to you, we've put in what we used
    but you can add/remove as needed
    return: 
    list of lists in the format [[x1, x2, ... , x(n-1), y], ...]
    '''

    # initializes an empty list to store the generated training samples
    samples = []
    
    # loops through the encoded data and generates the ngrams from the current list
    for e in encoded: 
        temp_list = []
        
        for i in range(len(e) - ngram + 1):
            temp_list.append(e[i:i+ngram])
        
        # adds the generated ngrams from the current list to the samples list 
        samples.extend(temp_list)
    
    return samples

# generates ngram training samples for both char and word tokenization data
char_samples = generate_ngram_training_samples(encoded=char_encoded, ngram=NGRAM)
word_samples = generate_ngram_training_samples(encoded=word_encoded, ngram=NGRAM)

print(len(char_samples))
print(len(word_samples))

# generate your training samples for both word and character data
# print out the first 5 training samples for each
# we have displayed the number of sequences
# to expect for both characters and words
#
# Spooky data by character should give 2957553 sequences
# [21, 21, 3]
# [21, 3, 9]
# [3, 9, 7]
# ...
# Spooky data by words should give 634080 sequences
# [1, 1, 32]
# [1, 32, 2956]
# [32, 2956, 3]
# ...

2957553
634080


### c) Then, split the sequences into X and y and create a Data Generator (20 points)

In [7]:
# 2.5 points

# Note here that the sequences were in the form: 
# sequence = [x1, x2, ... , x(n-1), y]
# We still need to separate it into [[x1, x2, ... , x(n-1)], ...], [y1, y2, ...]]
# do that here

def transform_seq_dimensions(samples: list):
    '''
    Takes the generated ngram training samples and
    splits them into X and y.
    Parameters:
    samples : list of training samples
    return: 
    X and y
    '''
    
    # initializes both X and y to be empty lists at first
    X = []
    y = []

    # for every sample, appends all elements of the current sample except for the last one to X,
    # and appends the last element of the current sample to y  
    for s in samples:
        X.append(s[:-1])
        y.append(s[-1])

    return X, y

# transforms both the char and word data into X and y
X_char, y_char = transform_seq_dimensions(char_samples)
X_word, y_word = transform_seq_dimensions(word_samples)

print(np.array(X_word).shape)
print(np.array(y_word).shape)
print(np.array(X_char).shape)
print(np.array(y_char).shape)

(634080, 2)
(634080,)
(2957553, 2)
(2957553,)


In [8]:
# 2.5 points

# Initialize a function that reads the word embeddings you saved earlier
# and gives you back mappings from words to their embeddings and also 
# indexes from the tokenizers to their embeddings

def read_embeddings(filename: str, tokenizer: Tokenizer) -> (dict, dict):
    '''Loads and parses embeddings trained in earlier.
    Parameters:
        filename (str): path to file
        Tokenizer: tokenizer used to tokenize the data (needed to get the word to index mapping)
    Returns:
        (dict): mapping from word to its embedding vector
        (dict): mapping from index to its embedding vector
    '''
    
    # loads the embeddings from the specified file
    embedding = KeyedVectors.load_word2vec_format(filename, binary=False)

    # initializes empty dictionaries to store the mappings
    word_to_vector = {}
    index_to_vector = {}
    
    # iterates through each index and word in the tokenizer's index_word mapping
    for index, word in tokenizer.index_word.items():
        # gets the vector that corresponds to the current word from the embeddings
        vector = embedding[word]
        # updates the word_to_vector dictionary with the current word and its vector
        word_to_vector[word] = vector
        # updates the index_to_vector dictionary with the current index and its vector
        index_to_vector[index] = vector

    return word_to_vector, index_to_vector

# reads the word embeddings from earlier and gives back their mappings
word_to_vector, word_index_to_vector = read_embeddings(EMBEDDING_SAVE_FILE_WORD, word_tokenizer)
char_to_vector, char_index_to_vector = read_embeddings(EMBEDDING_SAVE_FILE_CHAR, char_tokenizer)

In [9]:
# NECESSARY FOR CHARACTERS

# the "0" index of the Tokenizer is assigned for the padding token. Initialize
# the vector for padding token as all zeros of embedding size
# this adds one to the number of embeddings that were initially saved
# (and increases your vocab size by 1)

# initializes an empty list for the padding tokens
padding_token_vector = []

# appends a 0 to the padding_token_vector for each iteration
for x in range(EMBEDDINGS_SIZE):
    padding_token_vector.append(0)

# sets the value of the 0th index in the char_index_to_vector and word_index_to_vector
# to the padding_token_vector
char_index_to_vector[0] = padding_token_vector
word_index_to_vector[0] = padding_token_vector

# gets the keys of the char_index_to_vector dictionary
char_index_to_vector.keys()

dict_keys([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 0])

In [10]:
# 10 points

def data_generator(X: list, y: list, num_sequences_per_batch: int, index_2_embedding: dict) -> (list, list):
    '''
    Returns data generator to be used by feed_forward
    https://wiki.python.org/moin/Generators
    https://realpython.com/introduction-to-python-generators/
    
    Yields batches of embeddings and labels to go with them.
    Use one hot vectors to encode the labels 
    (see the to_categorical function)
    
    If for_feedforward is True: 
    Returns data generator to be used by feed_forward
    else: Returns data generator for RNN model
    '''
    
    # initializes the index for iterating through the data
    index = 0
    
    while True:

        # this is the data in the form [[21, 21], [21, 3], [3, 9], ...]
        embeddings = []
        
        # sets the starting and ending points for the current batch
        start = index
        end = min(index+num_sequences_per_batch, len(X))

        # loops through the indices in X for the current batch
        for indices in X[start:end]:
            # initializes an empty list to store the current row of embeddings
            row = []
            
            # extends the row with the corresponding embedding for each index  
            for i in indices:  
                row.extend(index_2_embedding[i])

            # appends the row to the embeddings list for the current batch
            embeddings.append(row)

        vocab_len = len(index_2_embedding.keys())
        
        # uses one-hot encoding to represent the class labels
        labels = to_categorical(y[start:end], num_classes=vocab_len)
        
        # reshape maintains the correct shape
        yield (np.array(embeddings).reshape(len(embeddings), EMBEDDINGS_SIZE*(NGRAM-1)), np.array(labels))
        
        # updates the index for the next batch and ensures it doesn't go over the length of X
        index += num_sequences_per_batch
        index %= len(X)

In [11]:
# 5 points

# initialize your data_generator for both word and character data
# print out the shapes of the first batch to verify that it is correct for both word and character data

# Examples:
# num_sequences_per_batch = 128 # this is the batch size
# steps_per_epoch = len(sequences)//num_sequences_per_batch # Number of batches per epoch
# train_generator = data_generator(X, y, num_sequences_per_batch)

# sample=next(train_generator) # this is how you get data out of generators
# sample[0].shape # (batch_size, (n-1)*EMBEDDING_SIZE) (128, 200)
# sample[1].shape # (batch_size, |V|) to_categorical

# word data
num_sequences_per_batch = 128 # this is the batch size
steps_per_epoch_word = len(X_word)//num_sequences_per_batch # Number of batches per epoch
train_generator_word = data_generator(X_word, y_word, num_sequences_per_batch, word_index_to_vector)

sample = next(train_generator_word) # this is how you get data out of generators

print(sample[0].shape)
print(sample[1].shape)

#sample[0].shape # (batch_size, (n-1)*EMBEDDING_SIZE) (128, 200)
#sample[1].shape # (batch_size, |V|) to_categorical

# character data
steps_per_epoch_char = len(X_char)//num_sequences_per_batch # Number of batches per epoch
train_generator_char = data_generator(X_char, y_char, num_sequences_per_batch, char_index_to_vector)

sample = next(train_generator_char) # this is how you get data out of generators

print(sample[0].shape)
print(sample[1].shape)

(128, 100)
(128, 25375)
(128, 100)
(128, 61)


### d) Train & __save__ your models (15 points)

In [12]:
# 15 points 

# code to train a feedforward neural language model for 
# both word embeddings and character embeddings
# make sure not to just copy + paste to train your two models
# (define functions as needed)

# train your models for between 3 & 5 epochs
# on Felix's machine, this takes ~ 24 min for character embeddings and ~ 10 min for word embeddings
# DO NOT EXPECT ACCURACIES OVER 0.5 (and even that is very for this many epochs)
# We recommend starting by training for 1 epoch

# Define your model architecture using Keras Sequential API
# Use the adam optimizer instead of sgd
# add cells as desired

from keras.optimizers import Adam

def build_nn(input_size: int, vocab_size: int) -> Sequential:
    '''
    Builds a neural network model.

    Parameters:
    input_size (int): The size of the input layer.
    vocab_size (int): The size of the vocabulary.

    Returns:
    Sequential: A Keras Sequential model.
    '''
    
    # initializes a Sequential model
    model = Sequential()

    # adds hidden layers
    model.add(Dense(units=128, activation="elu", input_shape=(input_size,)))
    model.add(Dense(units=64, activation="elu", input_shape=(input_size,)))

    # adds an output layer
    model.add(Dense(vocab_size, activation="softmax"))

    # shows the model's verbose
    model.summary()

    # calls compile here
    model.compile(loss='categorical_crossentropy',
                  optimizer="adam",
                  metrics=['accuracy'])

    return model

# builds the word and character neural models
word_model = build_nn((NGRAM-1) * EMBEDDINGS_SIZE, len(word_index_to_vector.keys()))
char_model = build_nn((NGRAM-1) * EMBEDDINGS_SIZE, len(char_index_to_vector.keys()))

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               12928     
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dense_2 (Dense)             (None, 25375)             1649375   
                                                                 
Total params: 1670559 (6.37 MB)
Trainable params: 1670559 (6.37 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 128)               12928     
                                                              

In [13]:
len(word_index_to_vector.keys())

25375

In [14]:
# Here is some example code to train a model with a data generator
# model.fit(x=train_generator, 
#           steps_per_epoch=steps_per_epoch,
#           epochs=1)

word_model.fit(x=train_generator_word, 
               steps_per_epoch=steps_per_epoch_word,
               epochs=5)

Epoch 1/5
4953/4953 [==============================] - 56s 11ms/step - loss: 5.6920 - accuracy: 0.1913
Epoch 2/5
4953/4953 [==============================] - 58s 12ms/step - loss: 5.2926 - accuracy: 0.2004
Epoch 3/5
4953/4953 [==============================] - 61s 12ms/step - loss: 5.1535 - accuracy: 0.2023
Epoch 4/5
4953/4953 [==============================] - 65s 13ms/step - loss: 5.0632 - accuracy: 0.2033
Epoch 5/5
4953/4953 [==============================] - 59s 12ms/step - loss: 5.0002 - accuracy: 0.2038


In [15]:
char_model.fit(x=train_generator_char, 
               steps_per_epoch=steps_per_epoch_char,
               epochs=5)

Epoch 1/5
23105/23105 [==============================] - 28s 1ms/step - loss: 2.0282 - accuracy: 0.3802
Epoch 2/5
23105/23105 [==============================] - 28s 1ms/step - loss: 1.9734 - accuracy: 0.3878
Epoch 3/5
23105/23105 [==============================] - 28s 1ms/step - loss: 1.9642 - accuracy: 0.3889
Epoch 4/5
23105/23105 [==============================] - 28s 1ms/step - loss: 1.9595 - accuracy: 0.3895
Epoch 5/5
23105/23105 [==============================] - 28s 1ms/step - loss: 1.9566 - accuracy: 0.3898


In [16]:
# spooky data model by character for 5 epochs takes ~ 24 min on Felix's computer
# with adam optimizer, gets accuracy of 0.3920

# spooky data model by word for 5 epochs takes 10 min on Felix's computer
# results in accuracy of 0.2110

In [17]:
# save your trained models so you can re-load instead of re-training each time
# also, you'll need these to generate your sentences!
char_model.save("char_model.h5")
word_model.save("word_model.h5")

/Users/maxnbf/anaconda3/lib/python3.11/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


### e) Generate Sentences (15 points)

In [18]:
# load your models if you need to
word_model = keras.models.load_model("word_model.h5")
char_model = keras.models.load_model("char_model.h5")

In [19]:
# 10 points

# generate a sequence from the model until you get an end of sentence token
# This is an example function header you might use

def generate_seq(model: Sequential, tokenizer: Tokenizer, seed: list, token_to_vector):
    '''
    Parameters:
        model: your neural network
        tokenizer: the keras preprocessing tokenizer
        seed: [w1, w2, w(n-1)]
    Returns: string sentence
    '''
    
    # copies the contents of the seed list to generated_sequence
    generated_sequence = seed.copy()

    while True:
        # selects the last two elements from the generated_sequence list
        words = generated_sequence[-2:]

        # assuming word_2_embedding is a pre-computed dictionary
        vectors = []
        for word in words:
            # appends the corresponding vector representation of the word from token_to_vector 
            # to the vectors list
            vectors.append(token_to_vector[word])

        # converts the vectors list to an array
        input_vec = np.array(vectors)

        # adjusts input dimensions accordingly
        test_sample = np.reshape(input_vec, [1, -1])

        # predicts the next word
        prediction = model.predict(test_sample, verbose=0)

        # randomly chooses an index from a range determined by the length of the tokenizer.word_index + 1
        # using the probabilities from the prediction array
        index = np.random.choice(range(len(tokenizer.word_index) + 1), p=prediction[0])
        
        # gets the next word that corresponds to the selected index from the tokenizer
        next_word = tokenizer.index_word[index]

        generated_sequence.append(next_word)

        # breaks the loop if next_word is equal to "</s>" 
        if next_word == "</s>":
            break

    # filters out the tokens "<s>" and "</s>" from the generated_sequence
    result = list(filter(lambda token: token not in ["<s>", "</s>"], generated_sequence))
        
    # joins all elements together to create the resulting generated sequence
    return " ".join(result)

In [20]:
# 5 points

# generate and display one sequence from both the word model and the character model
# do not include <s> or </s> in your displayed sentences
# make sure that you can read the output easily (i.e. don't just print out a list of tokens)

# you may leave _ as _ or replace it with a space if you prefer

print("Sentence generated using word token model:")
print(generate_seq(word_model, word_tokenizer, ["<s>", "<s>"], word_to_vector))

print("Sentence generated using character token model:")
char_sentence = generate_seq(char_model, char_tokenizer, ["<s>", "<s>"], char_to_vector)
char_sentence = char_sentence.replace(" ", "")
char_sentence = char_sentence.replace("_", " ")
print(char_sentence)

Sentence generated using word token model:
we had hurt .
Sentence generated using character token model:
holeand, thown.


In [21]:
# generate 100 example sentences with each model and save them to a file, one sentence per line
# do not include <s> and </s> in your saved sentences (you'll use these sentences in your next task)
# this will produce two files, one for each model

# names the file to save the word sentences to
word_model_file = "word_model_sentences.txt"

# writes 100 example sentences to the file
with open(word_model_file, "w", encoding="utf-8") as file:
    for _ in range(100):
        file.write(generate_seq(word_model, word_tokenizer, ["<s>", "<s>"], word_to_vector) + "\n")

In [23]:
# names the file to save the char sentences to
char_model_file = "char_model_sentences.txt"

# writes 100 example sentences to the file
with open(char_model_file, "w", encoding="utf-8") as file:
    for _ in range(100):
        char_sentence = generate_seq(char_model, char_tokenizer, ["<s>", "<s>"], char_to_vector)
        char_sentence = char_sentence.replace(" ", "")
        char_sentence = char_sentence.replace("_", " ")
        file.write(char_sentence + "\n")

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
